In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Сбор сравнительных таблиц по результатам из speed_tests/.
- Парсит файлы формата:
    root/speed_tests/<GPU_SETUP>/<MODEL>/<INPUTLEN>_length_<PARALLEL>_parallel.txt
- Извлекает "Average metrics ..." и блоки "Individual request metrics".
- Складывает результаты в:
    - speed_summary.csv                    — построчная таблица по каждому файлу
    - speed_summary_per_request.csv        — таблица по каждому индивидуальному запросу
    - speed_summary.md                     — удобная Markdown-версия результата
    - pivots/                              — несколько сводных CSV
"""

import re
import os
import csv
from pathlib import Path
from statistics import median
from collections import defaultdict

import pandas as pd

root_dir = "/root/llms/Qwen3_GPUS_metrics"
ROOT = Path(root_dir).resolve().parents[1]  # корень репозитория
SPEED_TESTS_DIR = ROOT / "speed_tests"
OUT_DIR = ROOT
PIVOT_DIR = OUT_DIR / "pivots"
PIVOT_DIR.mkdir(exist_ok=True)

# ---------- Регексы для парсинга содержимого ----------
# Общие средние метрики
AVG_LINE_PATTERNS = {
    "ttft": re.compile(r"\bttft:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
    "tokens_per_second": re.compile(r"\btokens_per_second:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
    "total_tokens": re.compile(r"\btotal_tokens:\s*([0-9]+)", re.I),
    "total_time": re.compile(r"\btotal_time:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
    "input_tokens": re.compile(r"\binput_tokens:\s*([0-9]+)", re.I),
    "output_tokens": re.compile(r"\boutput_tokens:\s*([0-9]+)", re.I),
    "concurrent_requests": re.compile(r"\bconcurrent_requests:\s*([0-9]+)", re.I),
    "valid_requests": re.compile(r"\bvalid_requests:\s*([0-9]+)", re.I),
    "total_concurrent_time": re.compile(r"\btotal_concurrent_time:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
}

# Шапка про средние для N/N
AVG_BLOCK_HEADER = re.compile(
    r"Average metrics for\s+(\d+)\s*/\s*(\d+)\s*concurrent requests.*", re.I
)

# Индивидуальные блоки:
REQUEST_BLOCK_RE = re.compile(
    r"Request\s+(\d+):\s*(.*?)\n(?=Request\s+\d+:|$)", re.I | re.S
)
REQ_FIELD_RE = {
    "ttft": re.compile(r"\bttft:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
    "tokens_per_second": re.compile(r"\btokens_per_second:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
    "total_tokens": re.compile(r"\btotal_tokens:\s*([0-9]+)", re.I),
    "total_time": re.compile(r"\btotal_time:\s*([0-9]+(?:\.[0-9]+)?)", re.I),
    "input_tokens": re.compile(r"\binput_tokens:\s*([0-9]+)", re.I),
    "output_tokens": re.compile(r"\boutput_tokens:\s*([0-9]+)", re.I),
}

# ---------- Вспомогательные функции ----------
def parse_path_meta(file_path: Path):
    """
    Извлекает GPU setup, модель, длину, параллелизм из пути.
    Пример: speed_tests/A2_x2/Qwen3-8B/5000_length_5_parallel.txt
    """
    gpu_setup = file_path.parts[-3]
    model = file_path.parts[-2]
    basename = file_path.stem  # '5000_length_5_parallel'
    m = re.match(r"(\d+)_length_(\d+)_parallel", basename)
    if not m:
        raise ValueError(f"Не удалось распарсить имя файла: {basename}")
    input_len = int(m.group(1))
    parallel = int(m.group(2))
    return gpu_setup, model, input_len, parallel

def parse_averages(text: str):
    data = {}
    # Пробуем вытащить "Average metrics for X/Y concurrent requests"
    m = AVG_BLOCK_HEADER.search(text)
    if m:
        data["avg_requests_ok"] = int(m.group(1))
        data["avg_requests_total"] = int(m.group(2))
    else:
        data["avg_requests_ok"] = None
        data["avg_requests_total"] = None

    for k, rx in AVG_LINE_PATTERNS.items():
        mm = rx.search(text)
        data[k] = float(mm.group(1)) if mm and '.' in mm.group(1) else (int(mm.group(1)) if mm else None)
    return data

def parse_per_request(text: str):
    per_rows = []
    for m in REQUEST_BLOCK_RE.finditer(text):
        req_id = int(m.group(1))
        block = m.group(2)
        row = {"req_id": req_id}
        for k, rx in REQ_FIELD_RE.items():
            mm = rx.search(block)
            row[k] = float(mm.group(1)) if mm and '.' in mm.group(1) else (int(mm.group(1)) if mm else None)
        per_rows.append(row)
    return sorted(per_rows, key=lambda r: r["req_id"])

# ---------- Основной проход по файлам ----------
file_rows = []
per_request_rows = []

for txt in SPEED_TESTS_DIR.rglob("*.txt"):
    try:
        gpu_setup, model, input_len, parallel = parse_path_meta(txt)
    except Exception:
        # пропускаем несоответствующие маске файлы
        continue

    content = txt.read_text(encoding="utf-8", errors="ignore")
    avg = parse_averages(content)
    req_rows = parse_per_request(content)

    base_meta = {
        "gpu_setup": gpu_setup,
        "model": model,
        "input_len": input_len,
        "parallel": parallel,
        "source_file": str(txt.relative_to(ROOT)),
    }

    # Строка со средними метриками
    row = {**base_meta, **avg}
    # Доп. удобные поля
    if avg.get("total_tokens") and avg.get("total_time"):
        row["avg_throughput_tokens_s"] = float(avg["total_tokens"]) / float(avg["total_time"]) if float(avg["total_time"]) > 0 else None
    else:
        row["avg_throughput_tokens_s"] = None

    # Медианы по индивидуальным запросам (если есть)
    if req_rows:
        for metric in ("ttft", "tokens_per_second", "total_time", "input_tokens", "output_tokens", "total_tokens"):
            vals = [r[metric] for r in req_rows if r.get(metric) is not None]
            row[f"median_{metric}"] = float(median(vals)) if vals else None

    file_rows.append(row)

    # Сохраняем по-запросно
    for r in req_rows:
        per_request_rows.append({**base_meta, **r})

# ---------- Выгрузка CSV ----------
df_files = pd.DataFrame(file_rows)
df_files.sort_values(["gpu_setup", "model", "input_len", "parallel"], inplace=True)
out_csv = OUT_DIR / "speed_summary.csv"
df_files.to_csv(out_csv, index=False)

df_reqs = pd.DataFrame(per_request_rows)
if not df_reqs.empty:
    df_reqs.sort_values(["gpu_setup", "model", "input_len", "parallel", "req_id"], inplace=True)
    out_csv_reqs = OUT_DIR / "speed_summary_per_request.csv"
    df_reqs.to_csv(out_csv_reqs, index=False)

# ---------- Пара готовых сводных таблиц (pivots) ----------
if not df_files.empty:
    # 1) Средний tokens_per_second по моделям и параллельности
    p1 = pd.pivot_table(
        df_files,
        index=["gpu_setup", "model"],
        columns="parallel",
        values="tokens_per_second",
        aggfunc="mean",
    )
    p1.to_csv(PIVOT_DIR / "pivot_tokens_per_second_by_parallel.csv")

    # 2) Средний ttft по длине входа (чем короче колонок — тем лучше)
    p2 = pd.pivot_table(
        df_files,
        index=["gpu_setup", "model"],
        columns="input_len",
        values="ttft",
        aggfunc="mean",
    )
    p2.to_csv(PIVOT_DIR / "pivot_ttft_by_inputlen.csv")

    # 3) Итоговый скользящий показатель throughput (avg_throughput_tokens_s)
    p3 = pd.pivot_table(
        df_files,
        index=["gpu_setup", "model"],
        columns="parallel",
        values="avg_throughput_tokens_s",
        aggfunc="mean",
    )
    p3.to_csv(PIVOT_DIR / "pivot_avg_throughput_by_parallel.csv")

# ---------- Markdown-версия основной таблицы ----------
md_path = OUT_DIR / "speed_summary.md"
with open(md_path, "w", encoding="utf-8") as f:
    f.write("# Speed Tests — Сводная таблица\n\n")
    if df_files.empty:
        f.write("_Данные не найдены в папке `speed_tests/`._\n")
    else:
        # Ограничим количество колонок для читабельности
        cols_show = [
            "gpu_setup", "model", "input_len", "parallel",
            "ttft", "tokens_per_second", "total_time",
            "input_tokens", "output_tokens", "total_tokens",
            "avg_requests_ok", "avg_requests_total",
            "avg_throughput_tokens_s",
            "median_ttft", "median_tokens_per_second", "median_total_time"
        ]
        cols_show = [c for c in cols_show if c in df_files.columns]
        f.write(df_files[cols_show].to_markdown(index=False))
        f.write("\n")

print(f"OK: {out_csv.name} создан.")
if not df_reqs.empty:
    print(f"OK: {out_csv_reqs.name} создан.")
print(f"OK: Markdown: {md_path.name}")
print(f"Сводные таблицы в: {PIVOT_DIR}")


KeyError: 'gpu_setup'